# Section 7: Result Extraction - Practice Questions

**Exam Weight**: 10% (~7 questions) | **Difficulty**: Medium | **Must Master**: ✅✅✅

---

## 🎯 Key Traps to Watch For:

| Trap | Wrong Assumption | Correct Understanding |
|------|------------------|----------------------|
| Result indexing | `result.data` | `result[0].data` - ALWAYS index first! |
| Sampler counts | `result.counts` | `result[0].data.meas.get_counts()` |
| Estimator values | `result.values` | `result[0].data.evs` - array of expectation values |
| Standard deviation | `result.stdev` | `result[0].data.stds` - plural! |
| Multiple circuits | One result | Iterate `result[i]` for each circuit |
| Bitstring order | Little-endian | Qiskit default, `reverse_bits()` for big-endian |

> 📖 See section_7_results/README.md for full concepts

---

## 📚 Topics Covered (from Section Notebooks):

### Result Extraction (`result_extraction.ipynb`)

#### PrimitiveResult Structure
```
PrimitiveResult
├── [0] PubResult (first PUB)
│   ├── .data
│   │   └── .meas (or register name) → BitArray
│   │       ├── .get_counts() → dict
│   │       ├── .get_bitstrings() → list
│   │       └── .num_shots
│   └── .metadata
├── [1] PubResult (second PUB)
└── ...
```

#### Sampler Results
- **Access pattern**: `result[pub_idx].data.<register>.get_counts()`
- **BitArray**: Container for measurement outcomes
- **Methods**: `.get_counts()`, `.get_bitstrings()`, `.num_shots`, `.num_bits`

#### Estimator Results
- **Expectation values**: `result[0].data.evs` → numpy array
- **Standard deviations**: `result[0].data.stds` → numpy array
- **Multiple observables**: `evs[obs_idx]` for each observable

#### Job Management
- **JobStatus enum**: `QUEUED`, `RUNNING`, `DONE`, `ERROR`, `CANCELLED`
- **Check status**: `job.status()` returns JobStatus
- **Retrieve job**: `service.job(job_id)` (singular!)
- **List jobs**: `service.jobs()` (plural!)

#### Advanced Features
- **Metadata access**: `result[0].metadata`
- **RuntimeEncoder/Decoder**: JSON serialization
- **Batch processing**: Multiple PUBs in single run

In [ ]:
# Setup - Run this first!
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.primitives import StatevectorSampler, StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp
import numpy as np
%matplotlib inline
print("✅ Setup complete!")

---
## Part 1: Sampler Result Hierarchy

```
PrimitiveResult
├── [0] PubResult (first PUB)
│   ├── .data
│   │   └── .meas (or register name) → BitArray
│   │       ├── .get_counts() → dict
│   │       ├── .num_shots
│   │       └── .num_bits
│   └── .metadata
├── [1] PubResult (second PUB)
└── ...
```

### Q1: Navigate Sampler result structure

In [ ]:
# Your solution: Run sampler and explore full result hierarchy

In [ ]:
# Solution Q1
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

sampler = StatevectorSampler()
job = sampler.run([(qc,)], shots=1000)
result = job.result()

print("=== Result Hierarchy ===")
print(f"Type: {type(result).__name__}")
print(f"Number of PUBs: {len(result)}")

pub_result = result[0]
print(f"\nPubResult type: {type(pub_result).__name__}")
print(f"PubResult.data: {pub_result.data}")

bit_array = pub_result.data.meas
print(f"\nBitArray type: {type(bit_array).__name__}")
print(f"  num_shots: {bit_array.num_shots}")
print(f"  num_bits: {bit_array.num_bits}")

counts = bit_array.get_counts()
print(f"\nCounts: {counts}")

### Q2: Multiple PUBs in result

In [ ]:
# Your solution: Run multiple circuits and access each result

In [ ]:
# Solution Q2
# Create different circuits
circuits = []
for n in [1, 2, 3]:
    qc = QuantumCircuit(n)
    qc.h(range(n))
    qc.measure_all()
    circuits.append(qc)

# Run all as PUBs
pubs = [(qc,) for qc in circuits]
result = StatevectorSampler().run(pubs, shots=100).result()

print(f"Total PUBs: {len(result)}")
for i in range(len(result)):
    counts = result[i].data.meas.get_counts()
    num_bits = result[i].data.meas.num_bits
    print(f"\nPUB {i}: {num_bits}-qubit circuit")
    print(f"  Outcomes: {len(counts)} unique states")

---
## Part 2: BitArray Methods

### Q3: BitArray properties and methods

In [ ]:
# Your solution: Explore all BitArray methods

In [ ]:
# Solution Q3
qc = QuantumCircuit(3)
qc.h([0, 1, 2])
qc.measure_all()

result = StatevectorSampler().run([(qc,)], shots=1000).result()
bit_array = result[0].data.meas

print("=== BitArray Properties ===")
print(f"num_bits: {bit_array.num_bits}")
print(f"num_shots: {bit_array.num_shots}")
print(f"shape: {bit_array.shape}")

print("\n=== BitArray Methods ===")
counts = bit_array.get_counts()
print(f"get_counts(): {counts}")

# get_int_counts returns int keys instead of bitstring keys
int_counts = bit_array.get_int_counts()
print(f"\nget_int_counts(): {int_counts}")

### Q4: Counts as integers vs bitstrings

In [ ]:
# Your solution: Compare get_counts() vs get_int_counts()

In [ ]:
# Solution Q4
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

result = StatevectorSampler().run([(qc,)], shots=100).result()
bit_array = result[0].data.meas

# String keys
str_counts = bit_array.get_counts()
print("get_counts() - string keys:")
for k, v in str_counts.items():
    print(f"  '{k}': {v}")

# Integer keys
int_counts = bit_array.get_int_counts()
print("\nget_int_counts() - integer keys:")
for k, v in int_counts.items():
    print(f"  {k} (binary: {bin(k)}): {v}")

---
## Part 3: Named Registers

### Q5: Access named classical registers

In [ ]:
# Your solution: Create circuit with named registers and access results

In [ ]:
# Solution Q5
qr = QuantumRegister(2, 'q')
cr_main = ClassicalRegister(2, 'main')
cr_flag = ClassicalRegister(1, 'flag')

qc = QuantumCircuit(qr, cr_main, cr_flag)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], cr_main)
qc.x(0)  # Flip for flag
qc.measure(0, cr_flag)

print(qc.draw())

result = StatevectorSampler().run([(qc,)], shots=100).result()

# Access by register name
pub_result = result[0]
print("\n=== Named Register Access ===")
print(f"'main' register: {pub_result.data.main.get_counts()}")
print(f"'flag' register: {pub_result.data.flag.get_counts()}")

### Q6: measure_all vs explicit measure

In [ ]:
# Your solution: Show register name difference between measure_all() and measure()

In [ ]:
# Solution Q6
# measure_all() creates 'meas' register
qc1 = QuantumCircuit(2)
qc1.h(0)
qc1.measure_all()  # Creates 'meas'

result1 = StatevectorSampler().run([(qc1,)]).result()
print("measure_all():")
print(f"  Register name: 'meas'")
print(f"  Counts: {result1[0].data.meas.get_counts()}")

# QuantumCircuit(n, m) creates 'c' register
qc2 = QuantumCircuit(2, 2)
qc2.h(0)
qc2.measure([0, 1], [0, 1])  # Uses default 'c'

result2 = StatevectorSampler().run([(qc2,)]).result()
print("\nExplicit measure to 'c':")
print(f"  Register name: 'c'")
print(f"  Counts: {result2[0].data.c.get_counts()}")

---
## Part 4: Estimator Results

### Q7: Estimator result structure

In [ ]:
# Your solution: Explore Estimator result hierarchy

In [ ]:
# Solution Q7
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)  # Bell state
# NO measurement for Estimator!

observable = SparsePauliOp('ZZ')

result = StatevectorEstimator().run([(qc, observable)]).result()

print("=== Estimator Result Hierarchy ===")
print(f"Type: {type(result).__name__}")
print(f"Number of PUBs: {len(result)}")

pub_result = result[0]
print(f"\nPubResult.data fields:")
print(f"  evs (expectation values): {pub_result.data.evs}")
print(f"  stds (standard deviations): {pub_result.data.stds}")

print(f"\nMetadata: {pub_result.metadata}")

### Q8: Multiple observables results

In [ ]:
# Your solution: Extract results for multiple observables

In [ ]:
# Solution Q8
qc = QuantumCircuit(1)
qc.h(0)  # |+⟩ state

observables = [
    SparsePauliOp('X'),
    SparsePauliOp('Y'),
    SparsePauliOp('Z')
]

# Create PUBs
pubs = [(qc, obs) for obs in observables]
result = StatevectorEstimator().run(pubs).result()

print("Expectation values for |+⟩:")
labels = ['X', 'Y', 'Z']
for i, label in enumerate(labels):
    evs = result[i].data.evs
    stds = result[i].data.stds
    print(f"  ⟨{label}⟩ = {evs:.4f} ± {stds:.4f}")

---
## Part 5: Metadata

### Q9: Access result metadata

In [ ]:
# Your solution: Explore metadata in results

In [ ]:
# Solution Q9
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

result = StatevectorSampler().run([(qc,)], shots=1000).result()

# Top-level metadata
print("=== Result Metadata ===")
print(f"Top-level metadata: {result.metadata}")

# PUB-level metadata
pub_result = result[0]
print(f"\nPUB metadata: {pub_result.metadata}")

# Estimator metadata
qc_est = QuantumCircuit(1)
qc_est.h(0)
est_result = StatevectorEstimator().run([(qc_est, SparsePauliOp('Z'))]).result()
print(f"\nEstimator metadata: {est_result[0].metadata}")

---
## Part 6: Common Exam Patterns

### Q10: Complete result extraction workflow

In [ ]:
# Your solution: Show both Sampler and Estimator extraction patterns

In [ ]:
# Solution Q10: Exam cheat sheet

print("="*50)
print("SAMPLER RESULT EXTRACTION")
print("="*50)

qc_samp = QuantumCircuit(2)
qc_samp.h(0)
qc_samp.cx(0, 1)
qc_samp.measure_all()

result_s = StatevectorSampler().run([(qc_samp,)], shots=100).result()

# Key paths:
print("\nKey paths:")
print(f"  result[0] → {type(result_s[0]).__name__}")
print(f"  result[0].data.meas → {type(result_s[0].data.meas).__name__}")
print(f"  result[0].data.meas.get_counts() → {result_s[0].data.meas.get_counts()}")

print("\n" + "="*50)
print("ESTIMATOR RESULT EXTRACTION")
print("="*50)

qc_est = QuantumCircuit(2)
qc_est.h(0)
qc_est.cx(0, 1)
# NO measurement!

result_e = StatevectorEstimator().run([(qc_est, SparsePauliOp('ZZ'))]).result()

# Key paths:
print("\nKey paths:")
print(f"  result[0] → {type(result_e[0]).__name__}")
print(f"  result[0].data.evs → {result_e[0].data.evs}")
print(f"  result[0].data.stds → {result_e[0].data.stds}")

---
## ✅ Section 7 Checklist

**Sampler Result Hierarchy**:
- [ ] `result[pub_idx]` → PubResult
- [ ] `result[i].data.meas` → BitArray (for measure_all)
- [ ] `result[i].data.<name>` → BitArray (named register)

**BitArray Methods**:
- [ ] `.get_counts()` → dict with string keys
- [ ] `.get_int_counts()` → dict with int keys
- [ ] `.num_shots`, `.num_bits`

**Estimator Result Hierarchy**:
- [ ] `result[pub_idx]` → PubResult
- [ ] `result[i].data.evs` → expectation value(s)
- [ ] `result[i].data.stds` → standard deviation(s)

**Register Names**:
- [ ] `measure_all()` → creates 'meas'
- [ ] `QuantumCircuit(n, m)` → creates 'c'
- [ ] Custom `ClassicalRegister(n, 'name')` → uses 'name'

**Metadata**:
- [ ] `result.metadata` → top-level
- [ ] `result[i].metadata` → per-PUB